In [0]:
def load_dataset(batch_size):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    train_dataset = torchvision.datasets.ImageFolder(
        root='/content/UTKFace/',
        transform=transform
    )
    # Split into train and validation
    np.random.seed(1000) # Fixed numpy random seed for reproducible shuffling
    indices = torch.randperm(len(train_dataset))
    print(len(train_dataset))
    train_indices, val_indices = indices[:16595], indices[16595:20151] #864 images are 80% of train folder

    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              num_workers=0, sampler=train_sampler)

    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              num_workers=0, sampler=val_sampler)

    test_dataset = torchvision.datasets.ImageFolder(root='/content/UTKFace', transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             num_workers=0, shuffle=True)
    return train_loader, val_loader, test_loader

In [0]:

import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [0]:
class conv_net(nn.Module):
    def __init__(self):
        super(conv_net, self).__init__()
        self.name = "cnn"
        self.conv1 = nn.Conv2d(3, 12, 5) 
        self.pool = nn.MaxPool2d(5, 5)
        self.conv2 = nn.Conv2d(12, 48, 5)
        self.fc1 = nn.Linear(2352, 2352)
        self.fc2 = nn.Linear(2352, 1024)
        self.fc3 = nn.Linear(1024, 256)
        self.fc4 = nn.Linear(256, 1)
    def forward(self, x):
        x = x.cuda()
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        x = x.squeeze(1)
        
        return x

In [0]:
def train_net(net, batch_size=32, learning_rate=0.008, num_epochs=10):
    ########################################################################
    # Fixed PyTorch random seed for reproducible result
    from torch.autograd import Variable
    torch.manual_seed(1000)
    ########################################################################
    # Obtain the PyTorch data loader objects to load batches of the datasets
    train_loader, val_loader, test_loader = load_dataset(batch_size)
    ########################################################################
    # Define the Loss function and optimizer
    # The loss function will be 
    # Optimizer will be SGD with Momentum.
    criterion = nn.MSELoss().cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

    # optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    start_time = time.time()
    iters, losses, train_acc, val_acc = [], [], [], []
    n = 0
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        total_epoch = 0

        for i, data in enumerate(train_loader, 0):
            # Get the inputs
            inputs, labels = data
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass, backward pass, and optimize

            outputs = net(Variable(inputs))
            loss = criterion(outputs.cuda(), labels.float().cuda())
            loss.backward()
            optimizer.step()

            total_epoch += len(labels)
            n+=1
            
        # save the current training information
        iters.append(n)
        losses.append(float(loss)/batch_size)             # compute *average* loss
        train_acc.append(get_accuracy(net, train_loader)) # compute training accuracy 
        val_acc.append(get_accuracy(net, val_loader))  # compute validation accuracy
            
        print("Epoch: {}, Training Loss: {:.3f}, Training R^2: {:.3f}, Validation R^2: {:.3f}".format(epoch, losses[-1],train_acc[-1],val_acc[-1]))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
        
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

In [0]:
def get_accuracy(model, data):
    c=0
    mean = 0.0
    for imgs, labels in data:
      mean += labels.sum()
      c+=32
    mean = (mean/c)
    
    #print(mean)
    
    correct = 0
    total = 0
    count = 0 
    ss_reg = 0
    ss_total = 0
    
    for imgs, labels in data:
        labels = labels.float()
        output = model(imgs) # We don't need to run F.softmax
        # print(output)
        # pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        # correct += pred.eq(labels.view_as(pred)).sum().item()
        output = torch.round(output)
        output = output.float()
        output = output.cpu().detach().numpy()
        output = torch.tensor(output)
        #correct += np.isclose(output.detach().numpy(), labels, 0.05).sum()
        total += imgs.shape[0]
        count+=1
        ss_reg += ((labels-output)**2).sum()
        ss_total += ((labels-mean.float())**2).sum()
    return 1-ss_reg/ss_total

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [97]:
CNN = conv_net()
use_cuda = True


CNN.cuda()
  
# ... Training ... 
#CNN.load_state_dict(torch.load("/content/" + get_model_name("cnn", 32,0.005, 9)))
CNN.load_state_dict(torch.load("/content/" + get_model_name("cnn", 32,1e-5, 3)))
train_net(CNN, batch_size=32, learning_rate=1e-7, num_epochs=20)

23708
Epoch: 0, Training Loss: 2.023, Training Accuracy: 0.767, Validation Accuracy: 0.493
Epoch: 1, Training Loss: 2.263, Training Accuracy: 0.767, Validation Accuracy: 0.493
Epoch: 2, Training Loss: 2.197, Training Accuracy: 0.767, Validation Accuracy: 0.493
Epoch: 3, Training Loss: 4.562, Training Accuracy: 0.767, Validation Accuracy: 0.493
Epoch: 4, Training Loss: 8.318, Training Accuracy: 0.767, Validation Accuracy: 0.493


KeyboardInterrupt: ignored

In [0]:
model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
torch.save(CNN.state_dict(), model_path)


In [0]:
CNN = conv_net()
CNN.load_state_dict(torch.load("/content/" + get_model_name("cnn", 32,0.0008, 10)))

In [95]:
p=0
train_loader, val_loader, test_loader = load_dataset(32)
freq = np.zeros(150)
for img, label in val_loader:
  out = CNN(img)
  out = out.float()
  label = label.cuda()
  
  for i in range(0,len(label)):
    freq[abs(label[i] - out[i].long())] +=1
    
freq.sum()

23708


3556.0

In [96]:
freq.astype(int)

array([183, 346, 311, 322, 277, 232, 248, 219, 174, 148, 124, 121, 104,
        92,  84,  64,  68,  51,  49,  21,  40,  26,  15,  30,  18,  21,
        15,   6,  18,  10,   6,  11,   7,  10,   8,   9,   4,   5,   5,
         4,   4,   3,   2,   3,   3,   0,   3,   3,   3,   4,   2,   3,
         0,   0,   1,   1,   2,   2,   1,   1,   0,   0,   2,   2,   0,
         0,   1,   0,   0,   0,   0,   0,   0,   1,   1,   1,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [34]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 11.5 GB  I Proc size: 2.2 GB
GPU RAM Free: 9656MB | Used: 1785MB | Util  16% | Total 11441MB


In [23]:
torch.cuda.current_device()
torch.cuda.device_count()

1

In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileId = drive.CreateFile({'id': '0BxYys69jI14kYVM3aVhKS1VhRUk'}) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  # folder_data.zip
fileId.GetContentFile('UTKFace.tar.gz')  # Save Drive file as a local file

UTKFace.tar.gz


In [0]:
!apt-get install p7zip-full
!p7zip -d UTKFace.tar.gz
!tar -xvf UTKFace.tar.gz

In [0]:
drive = "/content/UTKFace/"
import os

i=0
for i in range(1,117):
  # define the name of the directory to be created
  path = drive + "{}".format(i)

  try:  
      os.mkdir(path)
  except OSError:  
      print ("Creation of the directory %s failed" % path)
  else:  
      print ("Successfully created the directory %s " % path)

In [0]:
import shutil
k=0
for root, dirs, files in os.walk(drive): 
   for file in files:
      path_file = os.path.join(root,file)
      age = file.split("_")[0]
      destination = drive + str(age) + "/" + file
      if(file.endswith("jpg")):
        shutil.move(path_file, destination)
        k+=1
        if k%100==0:
          print ("Copying {} to age folder {}".format(file, destination))